# Install Offline MMDetection

In [1]:
!pip install -qqq /kaggle/input/mmdetv3-env/archive/addict-2.4.0-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmengine-0.7.4-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmcv-2.0.0-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/weighted-boxes-fusion/ensemble_boxes-1.0.9-py3-none-any.whl

In [2]:
!pip install -qqq /kaggle/input/mmdetection-3-1-evn/src/mmdet-3.1.0-py3-none-any.whl

In [3]:
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import wandb
from PIL import Image
import gc

from glob import glob
import matplotlib.pyplot as plt


import base64
import numpy as np
from pycocotools import _mask as coco_mask
from typing import Text, Dict, Tuple
import zlib

import mmdet, mmcv, mmengine
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules
from mmdet.apis import init_detector, inference_detector
from mmengine.visualization import Visualizer


print(mmdet.__version__)
print(mmcv.__version__)
print(mmengine.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


3.1.0
2.0.0
0.7.4


In [4]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def encode_binary_mask(mask: np.ndarray) -> Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [5]:
def get_glomerulus_mask(annotations: Dict, mask_shape: Tuple = (512, 512)) -> np.ndarray:
    """ Converts glomerulus labels into boolean mask """
    mask = np.ones(shape=mask_shape, dtype=np.uint8)
    
    for annotation in annotations: 
        if annotation['type'] == 'glomerulus':            
            coords = np.array(annotation['coordinates'])
            cv2.fillPoly(mask, pts=coords, color=0)

    return mask.astype(bool)
    


In [6]:
def morphological_gradient(binary_mask):
    """Áp dụng phép toán Morphological Gradient trên binary mask.
    binary_mask: Binary mask có giá trị 0 hoặc 255.

    Returns:
    - gradient: Kết quả Morphological Gradient.
    """
    # Chuyển đổi binary mask về kiểu dữ liệu uint8 nếu cần thiết
    if binary_mask.dtype != np.uint8:
        binary_mask = binary_mask.astype(np.uint8)

    # Áp dụng phép toán Morphological Gradient
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Kích thước kernel
    gradient = cv2.morphologyEx(binary_mask, cv2.MORPH_GRADIENT, kernel)

    return gradient



In [7]:
def convert_to_binary_mask(mask):
    """Chuyển đổi mask sang dạng binary mask True/False.
    mask: Mask có giá trị 0/255.

    Returns:
    - binary_mask: Binary mask dạng True/False.
    """
    binary_mask = (mask > 0)
    return binary_mask

In [8]:
def convert_to_mask(binary_mask):
    """Chuyển đổi binary mask sang dạng mask 0/255.
    binary_mask: Binary mask dạng True/False.

    Returns:
    - mask: Mask có giá trị 0/255.
    """
    mask = binary_mask.astype(np.uint8) * 255
    return mask

In [9]:
%mkdir work_dir_test

In [10]:
import mmcv
from mmdet.apis import inference_detector, init_detector
import albumentations as A
import torch
# from ensemble_boxes import *
import itertools

In [11]:
import sys
import os
import math
import random
import numpy as np
import tensorflow as tf
import scipy.misc
import skimage.color
import skimage.io
import urllib.request
import shutil

# URL from which to download the latest COCO trained weights
COCO_MODEL_URL = "https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5"


############################################################
#  Bounding Boxes
############################################################

def extract_bboxes(mask):
    """Compute bounding boxes from masks.
    mask: [height, width, num_instances]. Mask pixels are either 1 or 0.

    Returns: bbox array [num_instances, (y1, x1, y2, x2)].
    """
    boxes = np.zeros([mask.shape[-1], 4], dtype=np.int32)
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        # Bounding box.
        horizontal_indicies = np.where(np.any(m, axis=0))[0]
        vertical_indicies = np.where(np.any(m, axis=1))[0]
        if horizontal_indicies.shape[0]:
            x1, x2 = horizontal_indicies[[0, -1]]
            y1, y2 = vertical_indicies[[0, -1]]
            # x2 and y2 should not be part of the box. Increment by 1.
            x2 += 1
            y2 += 1
        else:
            # No mask for this instance. Might happen due to
            # resizing or cropping. Set bbox to zeros
            x1, x2, y1, y2 = 0, 0, 0, 0
        boxes[i] = np.array([y1, x1, y2, x2])
    return boxes.astype(np.int32)


def compute_iou(box, boxes, box_area, boxes_area):
    """Calculates IoU of the given box with the array of the given boxes.
    box: 1D vector [y1, x1, y2, x2]
    boxes: [boxes_count, (y1, x1, y2, x2)]
    box_area: float. the area of 'box'
    boxes_area: array of length boxes_count.

    Note: the areas are passed in rather than calculated here for
          efficency. Calculate once in the caller to avoid duplicate work.
    """
    # Calculate intersection areas
    y1 = np.maximum(box[0], boxes[:, 0])
    y2 = np.minimum(box[2], boxes[:, 2])
    x1 = np.maximum(box[1], boxes[:, 1])
    x2 = np.minimum(box[3], boxes[:, 3])
    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    union = box_area + boxes_area[:] - intersection[:]
    iou = intersection / union
    return iou


def compute_overlaps(boxes1, boxes2):
    """Computes IoU overlaps between two sets of boxes.
    boxes1, boxes2: [N, (y1, x1, y2, x2)].

    For better performance, pass the largest set first and the smaller second.
    """
    # Areas of anchors and GT boxes
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])

    # Compute overlaps to generate matrix [boxes1 count, boxes2 count]
    # Each cell contains the IoU value.
    overlaps = np.zeros((boxes1.shape[0], boxes2.shape[0]))
    for i in range(overlaps.shape[1]):
        box2 = boxes2[i]
        overlaps[:, i] = compute_iou(box2, boxes1, area2[i], area1)
    return overlaps


def compute_overlaps_masks(masks1, masks2):
    '''Computes IoU overlaps between two sets of masks.
    masks1, masks2: [Height, Width, instances]
    '''
    # flatten masks
    masks1 = np.reshape(masks1 > .5, (-1, masks1.shape[-1])).astype(np.float32)
    masks2 = np.reshape(masks2 > .5, (-1, masks2.shape[-1])).astype(np.float32)
    area1 = np.sum(masks1, axis=0)
    area2 = np.sum(masks2, axis=0)

    # intersections and union
    intersections = np.dot(masks1.T, masks2)
    union = area1[:, None] + area2[None, :] - intersections
    overlaps = intersections / union

    return overlaps


def non_max_suppression(boxes, scores, threshold):
    """Performs non-maximum supression and returns indicies of kept boxes.
    boxes: [N, (y1, x1, y2, x2)]. Notice that (y2, x2) lays outside the box.
    scores: 1-D array of box scores.
    threshold: Float. IoU threshold to use for filtering.
    """
    assert boxes.shape[0] > 0
    if boxes.dtype.kind != "f":
        boxes = boxes.astype(np.float32)

    # Compute box areas
    y1 = boxes[:, 0]
    x1 = boxes[:, 1]
    y2 = boxes[:, 2]
    x2 = boxes[:, 3]
    area = (y2 - y1) * (x2 - x1)

    # Get indicies of boxes sorted by scores (highest first)
    ixs = scores.argsort()[::-1]

    pick = []
    while len(ixs) > 0:
        # Pick top box and add its index to the list
        i = ixs[0]
        pick.append(i)
        # Compute IoU of the picked box with the rest
        iou = compute_iou(boxes[i], boxes[ixs[1:]], area[i], area[ixs[1:]])
        # Identify boxes with IoU over the threshold. This
        # returns indicies into ixs[1:], so add 1 to get
        # indicies into ixs.
        remove_ixs = np.where(iou > threshold)[0] + 1
        # Remove indicies of the picked and overlapped boxes.
        ixs = np.delete(ixs, remove_ixs)
        ixs = np.delete(ixs, 0)
    return np.array(pick, dtype=np.int32)


def apply_box_deltas(boxes, deltas):
    """Applies the given deltas to the given boxes.
    boxes: [N, (y1, x1, y2, x2)]. Note that (y2, x2) is outside the box.
    deltas: [N, (dy, dx, log(dh), log(dw))]
    """
    boxes = boxes.astype(np.float32)
    # Convert to y, x, h, w
    height = boxes[:, 2] - boxes[:, 0]
    width = boxes[:, 3] - boxes[:, 1]
    center_y = boxes[:, 0] + 0.5 * height
    center_x = boxes[:, 1] + 0.5 * width
    # Apply deltas
    center_y += deltas[:, 0] * height
    center_x += deltas[:, 1] * width
    height *= np.exp(deltas[:, 2])
    width *= np.exp(deltas[:, 3])
    # Convert back to y1, x1, y2, x2
    y1 = center_y - 0.5 * height
    x1 = center_x - 0.5 * width
    y2 = y1 + height
    x2 = x1 + width
    return np.stack([y1, x1, y2, x2], axis=1)


def box_refinement_graph(box, gt_box):
    """Compute refinement needed to transform box to gt_box.
    box and gt_box are [N, (y1, x1, y2, x2)]
    """
    box = tf.cast(box, tf.float32)
    gt_box = tf.cast(gt_box, tf.float32)

    height = box[:, 2] - box[:, 0]
    width = box[:, 3] - box[:, 1]
    center_y = box[:, 0] + 0.5 * height
    center_x = box[:, 1] + 0.5 * width

    gt_height = gt_box[:, 2] - gt_box[:, 0]
    gt_width = gt_box[:, 3] - gt_box[:, 1]
    gt_center_y = gt_box[:, 0] + 0.5 * gt_height
    gt_center_x = gt_box[:, 1] + 0.5 * gt_width

    dy = (gt_center_y - center_y) / height
    dx = (gt_center_x - center_x) / width
    dh = tf.log(gt_height / height)
    dw = tf.log(gt_width / width)

    result = tf.stack([dy, dx, dh, dw], axis=1)
    return result


def box_refinement(box, gt_box):
    """Compute refinement needed to transform box to gt_box.
    box and gt_box are [N, (y1, x1, y2, x2)]. (y2, x2) is
    assumed to be outside the box.
    """
    box = box.astype(np.float32)
    gt_box = gt_box.astype(np.float32)

    height = box[:, 2] - box[:, 0]
    width = box[:, 3] - box[:, 1]
    center_y = box[:, 0] + 0.5 * height
    center_x = box[:, 1] + 0.5 * width

    gt_height = gt_box[:, 2] - gt_box[:, 0]
    gt_width = gt_box[:, 3] - gt_box[:, 1]
    gt_center_y = gt_box[:, 0] + 0.5 * gt_height
    gt_center_x = gt_box[:, 1] + 0.5 * gt_width

    dy = (gt_center_y - center_y) / height
    dx = (gt_center_x - center_x) / width
    dh = np.log(gt_height / height)
    dw = np.log(gt_width / width)

    return np.stack([dy, dx, dh, dw], axis=1)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# WBF

In [12]:
import warnings
import numpy as np
from numba import jit

@jit(nopython=True)
def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

#@jit(nopython=True)
def get_weighted_mask(masks, scores, inmodels, conf_type):
    mask = np.zeros(masks[0].shape, dtype=np.float32)
    conf = 0
    conf_list = []
    for m, s, im in zip(masks, scores, inmodels):
        if conf_type == 'model_weight2':
            mask += s * im * m
            conf += s * im
        else:
            mask += s * m
            conf += s
        conf_list.append(s)
    score = np.max(conf_list)
    mask = mask / conf
    return mask, score, conf_list

def get_weighted_box(boxes, scores, inmodels, conf_type):
    box = np.zeros(4, dtype=np.float32)
    conf = 0
    conf_list = []
    for b, s, im in zip(boxes, scores, inmodels):
        if conf_type == 'model_weight2':
            box += s * im * b
            conf += s * im
        else:
            box += s * b
            conf += s
        conf_list.append(s)
    score = np.max(conf_list)
    box = box / conf
    return box, score


def find_matching_box(boxes_list, new_box, match_iou):
    best_iou = match_iou
    best_index = -1
    for i in range(len(boxes_list)):
        box = boxes_list[i]
        iou = bb_intersection_over_union(box, new_box)
        if iou > best_iou:
            best_index = i
            best_iou = iou

    return best_index, best_iou

def weighted_masks_fusion(masks, boxes, scores, models, iou_thr=0.7, skip_mask_thr=0.0, 
                        conf_type='max_weight', soft_weight=5, thresh_type=None, model_weights=1,
                        num_thresh=4, num_models=5):
    masks = masks[scores > skip_mask_thr]
    boxes = boxes[scores > skip_mask_thr]
    models = models[scores > skip_mask_thr]
    scores = scores[scores > skip_mask_thr]
    
    new_masks = []
    new_boxes = []
    new_scores = []
    inmodels = []
    weighted_boxes = []
    weighted_scores = []
    # Clusterize boxes
    for i in range(len(masks)):
            
        index, best_iou = find_matching_box(weighted_boxes, boxes[i], iou_thr)
        if index != -1:
            new_masks[index].append(masks[i])
            new_boxes[index].append(boxes[i])
            new_scores[index].append(scores[i])
            inmodels[index].append(models[i])
            weighted_boxes[index], weighted_scores[index] = get_weighted_box(new_boxes[index], new_scores[index], inmodels[index], conf_type)
        else:
            new_masks.append([masks[i]])
            new_boxes.append([boxes[i].copy()])
            new_scores.append([scores[i].copy()])
            inmodels.append([models[i]])
            weighted_boxes.append(boxes[i].copy())
            weighted_scores.append(scores[i].copy())
            
    ens_masks = []
    ens_scores = []
    ens_boxes = []
    for nmasks, nscores, wbox, inms in zip(new_masks, new_scores, weighted_boxes, inmodels):
        mask, score, conf_list = get_weighted_mask(nmasks, nscores, inms, conf_type)
        if thresh_type == 'num_thresh':
            if len(conf_list) >= num_thresh:
                ens_masks.append(mask)
                ens_boxes.append(wbox)
            else:
                continue
        else:
            ens_masks.append(mask)
            ens_boxes.append(wbox)

        if conf_type =='max_weight':
            ens_scores.append(score * min(len(conf_list), num_models) / num_models)
        elif conf_type == 'max':
            ens_scores.append(score)
        elif conf_type == 'soft_weight':
            ens_scores.append(score * (min(len(conf_list), num_models) + soft_weight) / (soft_weight + num_models))
        elif conf_type == 'model_weight' or conf_type == 'model_weight2':
            this_weights = [model_weights[i] for i in inms]
            ens_scores.append(score * (min(np.sum(this_weights), np.sum(model_weights)) + soft_weight) / (soft_weight + np.sum(model_weights)))

    return ens_masks, ens_scores, ens_boxes

In [13]:
def WTF(masks, boxes, scores, models):
    MODEL_WEIGHTS = [[1] * 4, [2] * 5 + [1] * 5, [2] * 5 + [1] * 5]
    # ensemble models using WMF. WMF is WBF applied to mask ensemble.
    pred_masks, pred_scores, pred_boxes = weighted_masks_fusion(masks, boxes, scores, models,
                                               skip_mask_thr=0.001,
                                               conf_type='max_weight',
                                               soft_weight=np.sum(MODEL_WEIGHTS[0]),
                                               num_models=5,
                                               model_weights = MODEL_WEIGHTS[0])
    
    return pred_masks, pred_scores, pred_boxes

In [14]:
def NMS(pred_masks, pred_scores, pred_boxes ):
    
    keep_ind = non_max_suppression(np.array(pred_boxes), np.array(pred_scores), 0.6)

    final_masks = np.array(pred_masks)[keep_ind,:, :]
    final_scores = np.array(pred_scores)[keep_ind]
    
    return final_masks, final_scores

In [15]:
def get_result(results,wbf=False):
    
    masks = []
    scores = []
    boxes = []
    labels = []
    models = []

    for i, result1 in enumerate(results):

        scores_1 = result1.pred_instances.scores.cpu().numpy()
        bboxes_1 = result1.pred_instances.bboxes.cpu().numpy()/513
        masks_1  = result1.pred_instances.masks.cpu().numpy()
        labels_1 = result1.pred_instances.labels.cpu().numpy()
        if i == 0:
            models = [0] * len(scores_1)
        else:
            models += [i] * len(scores_1)
        boxes.append(bboxes_1)
        scores.append(scores_1)
        masks.append(masks_1)
        labels.append(labels_1)
    
    masks = np.concatenate(masks, axis=0)
    scores = np.concatenate(scores, axis=0)
    boxes = np.concatenate(boxes, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    masks = np.array(masks)
    scores = np.array(scores)
    boxes = np.array(boxes)
    labels = np.array(labels)
    
    
    sort_idx = np.argsort(-scores)
    masks = masks[sort_idx]
    boxes = boxes[sort_idx]
    models = np.array(models)[sort_idx]
    scores = scores[sort_idx]
    
    print("len mask:", len(masks))
    print("len boxes:", len(boxes))
    print("len scores:", len(scores))
    print("len models:", len(models))
    
    pred_masks, pred_scores, pred_boxes = WTF(masks, boxes, scores, models)
    final_masks, final_scores = NMS(pred_masks, pred_scores, pred_boxes )

    return final_masks, final_scores

In [16]:
def get_mask(model_list,img):
    
    results = []
    for model in model_list:
        result_1 = inference_detector(model, imgs=img)
        results.append(result_1)
    
    final_masks, final_scores = get_result(results)

    
    return final_masks, final_scores

In [17]:
def load_model():
    base_cfg_path = "/kaggle/input/hubmap-best-weight/weight/base/fold1/custom_config.py"
    base_fold_1 = "/kaggle/input/hubmap-best-weight/weight/base/fold1/best_coco_segm_mAP.pth"
    base_fold_2 = "/kaggle/input/hubmap-best-weight/weight/base/fold2/best_coco_segm_mAP.pth"
    base_fold_3 = "/kaggle/input/hubmap-best-weight/weight/base/fold3/best_coco_segm_mAP.pth"
    base_fold_4 = "/kaggle/input/hubmap-best-weight/weight/base/fold4/best_coco_segm_mAP.pth"
    base_fold_5 = "/kaggle/input/hubmap-best-weight/weight/base/fold5/best_coco_segm_mAP.pth"

    pseudo_cfg_path = "/kaggle/input/hubmap-best-weight/weight/pseudo/fold1/custom_config.py"
    pseudo_fold_1 = "/kaggle/input/hubmap-best-weight/weight/pseudo/fold1/best_coco_segm_mAP.pth"
    pseudo_fold_2 = "/kaggle/input/hubmap-best-weight/weight/pseudo/fold2/best_coco_segm_mAP.pth"
    pseudo_fold_3 = "/kaggle/input/hubmap-best-weight/weight/pseudo/fold3/best_coco_segm_mAP.pth"
    pseudo_fold_4 = "/kaggle/input/hubmap-best-weight/weight/pseudo/fold4/best_coco_segm_mAP.pth"
    pseudo_fold_5 = "/kaggle/input/hubmap-best-weight/weight/pseudo/fold5/best_coco_segm_mAP.pth"
    pseudo_split  = "/kaggle/input/hubmap-best-weight/weight/pseudo/split/best_coco_segm_mAP.pth"

    tmp_cfg_path = "/kaggle/input/hubmap-best-weight/weight/tmp/fold1/custom_config.py"
    tmp_fold_1 = "/kaggle/input/hubmap-best-weight/weight/tmp/fold1/best_coco_segm_mAP.pth"
    tmp_fold_2 = "/kaggle/input/hubmap-best-weight/weight/tmp/fold2/best_coco_segm_mAP.pth"
    tmp_fold_3 = "/kaggle/input/hubmap-best-weight/weight/tmp/fold3/best_coco_segm_mAP.pth"
    tmp_fold_4 = "/kaggle/input/hubmap-best-weight/weight/tmp/fold4/best_coco_segm_mAP.pth"
    tmp_fold_5 = "/kaggle/input/hubmap-best-weight/weight/tmp/fold5/best_coco_segm_mAP.pth"
    tmp_split  = "/kaggle/input/hubmap-best-weight/weight/tmp/split/best_coco_segm_mAP.pth"

    aug_cfg_path = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/fold1/custom_config.py"
    aug_fold_1 = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/fold1/best_coco_segm_mAP.pth"
    aug_fold_2 = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/fold2/best_coco_segm_mAP.pth"
    aug_fold_3 = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/fold3/best_coco_segm_mAP.pth"
    aug_fold_4 = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/fold4/best_coco_segm_mAP.pth"
    aug_fold_5 = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/fold5/best_coco_segm_mAP.pth"
    aug_split  = "/kaggle/input/hubmap-best-weight/weight/tmp_aug/split/best_coco_segm_mAP.pth"

    hrnet_cfg_path = "/kaggle/input/hubmap-best-weight/weight/hrnet/fold1/custom_config.py"
    hrnet_fold_1 = "/kaggle/input/hubmap-best-weight/weight/hrnet/fold1/best_coco_segm_mAP.pth"
    
    base_cfg = Config.fromfile(base_cfg_path)
    base_cfg.model.test_cfg.rcnn.mask_thr_binary = 0.25
    base_cfg.model.test_cfg.rcnn.score_thr = 0.01

    pseudo_cfg = Config.fromfile(pseudo_cfg_path)
    pseudo_cfg.model.test_cfg.rcnn.mask_thr_binary = 0.25
    pseudo_cfg.model.test_cfg.rcnn.score_thr = 0.01

    tmp_cfg = Config.fromfile(tmp_cfg_path)
    tmp_cfg.model.test_cfg.rcnn.mask_thr_binary = 0.25
    tmp_cfg.model.test_cfg.rcnn.score_thr = 0.01

    aug_cfg = Config.fromfile(aug_cfg_path)
    aug_cfg.model.test_cfg.rcnn.mask_thr_binary = 0.25
    aug_cfg.model.test_cfg.rcnn.score_thr = 0.01

    hrnet_cfg = Config.fromfile(hrnet_cfg_path)
    hrnet_cfg.model.test_cfg.rcnn.mask_thr_binary = 0.25
    hrnet_cfg.model.test_cfg.rcnn.score_thr = 0.01

    model_1 = init_detector(base_cfg, checkpoint=base_fold_1, device="cuda:0")
#     model_2 = init_detector(base_cfg, checkpoint=base_fold_2, device="cuda:0")
    #model_3 = init_detector(base_cfg, checkpoint=base_fold_3, device="cuda:0")
#     model_4 = init_detector(base_cfg, checkpoint=base_fold_4, device="cuda:0")
#     model_5 = init_detector(base_cfg, checkpoint=base_fold_5, device="cuda:0")

    model_6  = init_detector(pseudo_cfg, checkpoint=pseudo_fold_1, device="cuda:0")
#     model_7  = init_detector(pseudo_cfg, checkpoint=pseudo_fold_2, device="cuda:0")
    #model_8  = init_detector(pseudo_cfg, checkpoint=pseudo_fold_3, device="cuda:0")
#     model_9  = init_detector(pseudo_cfg, checkpoint=pseudo_fold_4, device="cuda:0")
#     model_10 = init_detector(pseudo_cfg, checkpoint=pseudo_fold_5, device="cuda:0")
    model_11 = init_detector(pseudo_cfg, checkpoint=pseudo_split, device="cuda:0")

    model_12 = init_detector(tmp_cfg, checkpoint=tmp_fold_1, device="cuda:0")
#     model_13 = init_detector(tmp_cfg, checkpoint=tmp_fold_2, device="cuda:0")
    #model_14 = init_detector(tmp_cfg, checkpoint=tmp_fold_3, device="cuda:0")
#     model_15 = init_detector(tmp_cfg, checkpoint=tmp_fold_4, device="cuda:0")
#     model_16 = init_detector(tmp_cfg, checkpoint=tmp_fold_5, device="cuda:0")
    model_17 = init_detector(tmp_cfg, checkpoint=tmp_split, device="cuda:0")

    model_18 = init_detector(aug_cfg, checkpoint=aug_fold_1, device="cuda:0")
#     model_19 = init_detector(aug_cfg, checkpoint=aug_fold_2, device="cuda:0")
    #model_20 = init_detector(aug_cfg, checkpoint=aug_fold_3, device="cuda:0")
#     model_21 = init_detector(aug_cfg, checkpoint=aug_fold_4, device="cuda:0")
#     model_22 = init_detector(aug_cfg, checkpoint=aug_fold_5, device="cuda:0")
    model_23 = init_detector(aug_cfg, checkpoint=aug_split, device="cuda:0")

    model_24 = init_detector(hrnet_cfg, checkpoint=hrnet_fold_1, device="cuda:0")
    
    model_list = [
                model_1, 
                #         model_2,
                #model_3,
                #         model_4, model_5,

                model_6,
                #         model_7,
               # model_8,
                #         model_9, model_10,
                model_11,

                model_12,
                #         model_13,
               # model_14,
                #         model_15, model_16,
                model_17,

                model_18,
                #         model_19,
               # model_20,
                #         model_21, model_22,
                model_23,
                model_24
            ]
    
    return model_list

In [18]:
model_list = load_model()

Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/base/fold1/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/pseudo/fold1/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/pseudo/split/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/tmp/fold1/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/tmp/split/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/tmp_aug/fold1/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/tmp_aug/split/best_coco_segm_mAP.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-best-weight/weight/hrnet/fold1/best_coco_segm_mAP.pth


In [19]:
all_rows = []
results_csv = pd.DataFrame([], columns=["id","height","width","prediction_string"])
results_csv.set_index("id")

for dirname, _, filenames in os.walk('/kaggle/input/hubmap-hacking-the-human-vasculature/test'):
    
    for filename in filenames:
        
        img = mmcv.imread(os.path.join(dirname, filename))
        width, height, _ = img.shape
        
        row = dict()
        row["id"] = filename[:-4]
        row["height"] = height
        row["width"] = width
        row["prediction_string"] = ""

        mask,conf = get_mask(model_list, img)
                    
        for i in range(len(mask)):
            sliceImage = mask[i]
            sliceImage = np.where(sliceImage > 0.5, 1, 0)
            sliceImage = sliceImage*255
            
            kernel = np.ones(shape=(3, 3), dtype=np.uint8)
            sliceImage = cv2.dilate(sliceImage.astype(np.uint8), kernel, 4)
        
            coded_len = encode_binary_mask(sliceImage.astype(np.bool_)).decode('utf-8')
            row["prediction_string"] += '0 ' + str(conf[i])+' '+ coded_len+' '

        
        new_row = pd.DataFrame(row, index=[0])
        results_csv = pd.concat([new_row, results_csv.loc[:]]).reset_index(drop=True)

results_csv.to_csv("submission.csv", index=False)

/opt/conda/lib/python3.10/site-packages/mmdet/models/roi_heads/mask_heads/fcn_mask_head.py:339: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  mask_preds = bboxes.new_tensor(mask_preds)


len mask: 750
len boxes: 750
len scores: 750
len models: 750


/tmp/ipykernel_24/467368416.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if mask.dtype != np.bool:


In [20]:
results_csv

,id,height,width,prediction_string
0,72e40acccadf,512,512,0 0.9941443800926208 eNrLj8sySTCIs7fysHQ38Dc0w...
